In [ ]:
from skimage import io, color, morphology
from skimage.util import img_as_float, img_as_ubyte
import matplotlib.pyplot as plt
import numpy as np
import math
from skimage.filters import threshold_otsu
from skimage import segmentation
from skimage import measure
from skimage.color import label2rgb

In [ ]:
def show_comparison(original, modified, modified_name):
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(8, 4), sharex=True,
                                   sharey=True)
    ax1.imshow(original)
    ax1.set_title('Original')
    ax1.axis('off')
    ax2.imshow(modified)
    ax2.set_title(modified_name)
    ax2.axis('off')
    io.show()

Exercise 1

In [ ]:
img = io.imread("data/lego_4_small.png")

# Step 2: Convert to grayscale
gray = color.rgb2gray(img)

# Step 3: Threshold with Otsu's method
T = threshold_otsu(gray)
binary = gray > T

# Step 4: Show comparison
show_comparison(img, binary, 'Binary (Otsu Threshold)')

Exercise 2

In [ ]:
from skimage.segmentation import clear_border
cleaned = clear_border(binary)

# Show result
show_comparison(binary, cleaned, 'Without Border BLOBs')

Exercise 3

In [ ]:
from skimage.morphology import closing, opening, disk

# Create disk-shaped footprint
footprint = disk(5)

# Step 1: Apply closing to fill holes
closed = closing(cleaned, footprint)

# Step 2: Apply opening to remove noise
cleaned_morph = opening(closed, footprint)

# Show the cleaned result
show_comparison(cleaned, cleaned_morph, 'After Closing + Opening')


Exercise 4

In [ ]:
from skimage import measure

# Step 1: Label the cleaned binary image
label_img = measure.label(cleaned_morph)

# Step 2: Count number of labels (excluding background = 0)
n_labels = label_img.max()
print(f"Number of labels: {n_labels}")


Exercise 5

In [ ]:
from skimage.color import label2rgb

# Convert labeled image into a color overlay
# Use the original grayscale image as background (for clarity)
label_overlay = label2rgb(label_img, image=gray, bg_label=0)

# Show original and labeled overlay
show_comparison(img, label_overlay, "Labeled BLOBs (label2rgb)")


Exercise 6

In [ ]:
from skimage import measure
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Get region properties from labeled image
region_props = measure.regionprops(label_img)

# Step 2: Extract area of each region
areas = np.array([prop.area for prop in region_props])

# Step 3: Plot histogram of areas
plt.figure(figsize=(8, 4))
plt.hist(areas, bins=50, color='steelblue', edgecolor='black')
plt.title("Histogram of BLOB Areas")
plt.xlabel("Area (pixels)")
plt.ylabel("Count")
plt.grid(True)
plt.tight_layout()
plt.show()


Exercise 7

In [ ]:
!python Ex5-BlobAnalysisInteractive.py

In [ ]:
from skimage import io
import matplotlib.pyplot as plt

# Step 1: Load the 16-bit .tiff image
in_dir = "data/"
img_org = io.imread(in_dir + 'Sample E2 - U2OS DAPI channel.tiff')

# Step 2: Extract a smaller region of interest (ROI)
img_small = img_org[700:1200, 900:1400]

# Step 3: Convert 16-bit slice to 8-bit
img_gray = img_as_ubyte(img_small)

# Step 4: Visualize using limited intensity window
io.imshow(img_gray, vmin=0, vmax=150)
plt.title('DAPI Stained U2OS cell nuclei')
plt.axis("off")
io.show()

# Step 5: Inspect histogram to pick a good threshold
plt.figure(figsize=(6, 4))
plt.hist(img_gray.ravel(), bins=256, range=(1, 100))
plt.title("Histogram of grayscale intensities")
plt.xlabel("Gray value")
plt.ylabel("Pixel count")
plt.grid(True)
plt.tight_layout()
plt.show()


Exercise 8

In [ ]:
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
import matplotlib.pyplot as plt

# --- Assume img_gray is your 8-bit sliced grayscale image ---

# Option 1: Use Otsu's method
T = threshold_otsu(img_gray)
print(f"Otsu threshold value: {T}")

# Option 2: Manual threshold (uncomment to use a fixed one)
# T = 30  # You can tune this based on histogram

# Apply threshold
binary = img_gray > T

# Remove objects touching image border
binary_clean = clear_border(binary)

# Show original and binary result
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
ax[0].imshow(img_gray, cmap='gray', vmin=0, vmax=150)
ax[0].set_title("Original Grayscale Image")
ax[0].axis("off")

ax[1].imshow(binary_clean, cmap='gray')
ax[1].set_title(f"Binary Mask (Threshold = {T})")
ax[1].axis("off")

plt.tight_layout()
plt.show()


Exercise 9

In [ ]:
from skimage.segmentation import clear_border
from skimage import measure
from skimage.color import label2rgb
import matplotlib.pyplot as plt

# Assume you already have:
# img_gray (grayscale image)
# binary (thresholded binary image)

# Step 1: Remove border blobs
img_c_b = clear_border(binary)

# Step 2: Label the connected components
label_img = measure.label(img_c_b)

# Step 3: Overlay labels on original grayscale image
image_label_overlay = label2rgb(label_img, image=img_gray, bg_label=0)

# Step 4: Show the original image and the label overlay
def show_comparison(original, modified, modified_name):
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 5), sharex=True, sharey=True)
    ax1.imshow(original, cmap='gray', vmin=0, vmax=150)
    ax1.set_title('Original Grayscale')
    ax1.axis('off')
    ax2.imshow(modified)
    ax2.set_title(modified_name)
    ax2.axis('off')
    plt.tight_layout()
    plt.show()

show_comparison(img_gray, image_label_overlay, 'Found BLOBS')


Exercise 10

In [ ]:
from skimage import measure
import numpy as np

# Step 1: Compute region properties
region_props = measure.regionprops(label_img)

# Step 2: Print features of the first object
first_obj = region_props[0]
print(f"Object 1: Area = {first_obj.area}")
print(f"          Eccentricity = {first_obj.eccentricity}")
print(f"          Solidity = {first_obj.solidity}")
print(f"          Centroid = {first_obj.centroid}")


In [ ]:
valid_labels = []
min_area = 80  # adjust based on histogram or inspection
max_area = 1000

for i, prop in enumerate(region_props):
    if min_area < prop.area < max_area:
        valid_labels.append(i + 1)  # labels are 1-indexed

# Create a filtered binary mask of valid nuclei
filtered_mask = np.isin(label_img, valid_labels)

# Visualize result
from skimage.color import label2rgb
filtered_label_img = measure.label(filtered_mask)
overlay = label2rgb(filtered_label_img, image=img_gray, bg_label=0)

show_comparison(img_gray, overlay, "Filtered Nuclei")


Exercise 11

In [ ]:
from skimage import measure
import numpy as np

# Step 1: Choose thresholds (you can adjust these based on the area histogram)
min_area = 80
max_area = 1000

# Step 2: Copy the original label image
label_img_filter = label_img.copy()

# Step 3: Loop through region properties and remove invalid objects
for region in region_props:
    if region.area < min_area or region.area > max_area:
        for coords in region.coords:
            label_img_filter[coords[0], coords[1]] = 0

# Step 4: Create binary mask from filtered labels
i_area = label_img_filter > 0

# Step 5: Show result
show_comparison(img_small, i_area, 'Found nuclei based on area')


In [ ]:
areas = np.array([r.area for r in region_props])
plt.hist(areas, bins=50)
plt.title("Histogram of Nucleus Areas")
plt.xlabel("Area (pixels)")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()


Exercise 12

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from skimage import measure

# Recompute region_props in case label_img has been filtered
region_props = measure.regionprops(label_img)

# Step 1: Extract area and perimeter
areas = np.array([prop.area for prop in region_props])
perimeters = np.array([prop.perimeter for prop in region_props])

# Step 2: Scatter plot of area vs perimeter
plt.figure(figsize=(7, 5))
plt.scatter(areas, perimeters, alpha=0.7, edgecolor='k')
plt.title("Feature Space: Area vs. Perimeter")
plt.xlabel("Area (pixels)")
plt.ylabel("Perimeter (pixels)")
plt.grid(True)
plt.tight_layout()
plt.show()


Exercise 13

In [ ]:
from skimage import measure
import numpy as np
import matplotlib.pyplot as plt

# Recompute region properties if needed
region_props = measure.regionprops(label_img)

# Step 1: Compute circularity
circularities = []
for prop in region_props:
    area = prop.area
    perimeter = prop.perimeter
    if perimeter > 0:
        circ = 4 * np.pi * area / (perimeter ** 2)
    else:
        circ = 0
    circularities.append(circ)

circularities = np.array(circularities)

# Step 2: Plot histogram of circularity
plt.figure(figsize=(7, 4))
plt.hist(circularities, bins=50, color='skyblue', edgecolor='black')
plt.title("Histogram of Circularity")
plt.xlabel("Circularity")
plt.ylabel("Number of Blobs")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Define thresholds
min_area = 80
max_area = 1000
min_circ = 0.6
max_circ = 1.0

# Filter based on both area and circularity
label_img_combined = label_img.copy()
for i, prop in enumerate(region_props):
    area = prop.area
    perimeter = prop.perimeter
    circ = 4 * np.pi * area / (perimeter ** 2) if perimeter > 0 else 0
    if area < min_area or area > max_area or circ < min_circ or circ > max_circ:
        for coord in prop.coords:
            label_img_combined[coord[0], coord[1]] = 0

# Create final binary mask
i_combined = label_img_combined > 0
show_comparison(img_small, i_combined, 'Accepted nuclei (Area + Circularity)')


Exercise 14

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from skimage import measure

# Recompute region_props if needed
region_props = measure.regionprops(label_img)

# Extract area and circularity
areas = []
circularities = []

for prop in region_props:
    area = prop.area
    perimeter = prop.perimeter
    circ = 4 * np.pi * area / (perimeter ** 2) if perimeter > 0 else 0
    areas.append(area)
    circularities.append(circ)

areas = np.array(areas)
circularities = np.array(circularities)

# Plot Area vs Circularity
plt.figure(figsize=(7, 5))
plt.scatter(areas, circularities, alpha=0.7, edgecolor='k')
plt.title("Area vs. Circularity of Blobs")
plt.xlabel("Area (pixels)")
plt.ylabel("Circularity")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# Define criteria
min_area = 80
max_area = 1000
min_circ = 0.6

# Count valid nuclei
valid_nuclei = 0
for prop in region_props:
    area = prop.area
    perimeter = prop.perimeter
    circ = 4 * np.pi * area / (perimeter ** 2) if perimeter > 0 else 0
    if min_area <= area <= max_area and min_circ <= circ <= 1.0:
        valid_nuclei += 1

print(f"Number of well-formed nuclei: {valid_nuclei}")


Exercise 15

In [ ]:
from skimage import io , measure
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.morphology import closing, opening, disk
import numpy as np
import matplotlib.pyplot as plt

# --- Parameters ---
in_dir = "data/"
filename = 'Sample E2 - U2OS DAPI channel.tiff'
min_area = 80
max_area = 1000
min_circ = 0.6
radius = 3  # For closing/opening

# --- Step 1: Load full image (16-bit) and convert to 8-bit per slice ---
img_org = io.imread(in_dir + filename)

# Define regions (e.g., 3 horizontal, 2 vertical slices)
height, width = img_org.shape
rows = [(0, 700), (700, 1200), (1200, 1440)]
cols = [(0, 800), (800, 1600)]

# --- Step 2: Process each region ---
region_count = 0
total_nuclei = 0

for r_start, r_end in rows:
    for c_start, c_end in cols:
        region_count += 1

        # Slice region and convert to 8-bit
        img_small = img_org[r_start:r_end, c_start:c_end]
        img_gray = img_as_ubyte(img_small)

        # Threshold
        T = threshold_otsu(img_gray)
        binary = img_gray > T
        binary = clear_border(binary)

        # Morphological cleaning
        cleaned = opening(closing(binary, disk(radius)), disk(radius))

        # Label and extract props
        label_img = measure.label(cleaned)
        region_props = measure.regionprops(label_img)

        # Count valid nuclei
        count = 0
        for prop in region_props:
            area = prop.area
            perimeter = prop.perimeter
            circ = 4 * np.pi * area / (perimeter ** 2) if perimeter > 0 else 0
            if min_area <= area <= max_area and circ >= min_circ:
                count += 1

        total_nuclei += count
        print(f"Region {region_count}: ({r_start}:{r_end}, {c_start}:{c_end}) → Nuclei count: {count}")

print(f"\n✅ Total nuclei across all regions: {total_nuclei}")


Exercise 16

In [ ]:
from skimage import io, measure
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.morphology import closing, opening, disk
import numpy as np
import matplotlib.pyplot as plt

# Load COS7 image (16-bit)
img_cos7 = io.imread("data/Sample G1 - COS7 cells DAPI channel.tiff")

# Slice a region (you can test several)
img_small = img_cos7[700:1200, 900:1400]

# Convert to 8-bit
img_gray = img_as_ubyte(img_small)

# Threshold
T = threshold_otsu(img_gray)
binary = img_gray > T
binary = clear_border(binary)

# Morphological cleaning
footprint = disk(3)
cleaned = opening(closing(binary, footprint), footprint)

# Label and compute region properties
label_img = measure.label(cleaned)
region_props = measure.regionprops(label_img)

# Parameters
min_area = 80
max_area = 1000
min_circ = 0.6

# Count well-formed nuclei
valid_labels = []
for i, prop in enumerate(region_props):
    area = prop.area
    perimeter = prop.perimeter
    circ = 4 * np.pi * area / (perimeter ** 2) if perimeter > 0 else 0
    if min_area <= area <= max_area and circ >= min_circ:
        valid_labels.append(i + 1)  # label_img is 1-indexed

# Create final binary mask
final_mask = np.isin(label_img, valid_labels)

# Visualize result
from skimage.color import label2rgb

label_overlay = label2rgb(measure.label(final_mask), image=img_gray, bg_label=0)

def show_comparison(original, overlay, title):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    ax1.imshow(original, cmap='gray', vmin=0, vmax=150)
    ax1.set_title("Original COS7")
    ax1.axis("off")
    ax2.imshow(overlay)
    ax2.set_title(title)
    ax2.axis("off")
    plt.tight_layout()
    plt.show()

show_comparison(img_gray, label_overlay, f"Detected COS7 nuclei ({len(valid_labels)})")


Exercise 17

In [ ]:
from skimage.morphology import opening, disk
from skimage import measure
import numpy as np

# Step 1: Apply opening to separate weakly touching nuclei
separated = opening(cleaned, disk(2))

# Step 2: Label and recompute regionprops
label_img_sep = measure.label(separated)
region_props_sep = measure.regionprops(label_img_sep)

# Step 3: Filter by area and circularity
valid_labels = []
for i, prop in enumerate(region_props_sep):
    area = prop.area
    perimeter = prop.perimeter
    circ = 4 * np.pi * area / (perimeter ** 2) if perimeter > 0 else 0
    if min_area <= area <= max_area and circ >= min_circ:
        valid_labels.append(i + 1)

# Create final mask of separated, well-formed nuclei
final_mask = np.isin(label_img_sep, valid_labels)
